In [12]:
import csv
import math
import operator
import functools

import numpy as np
import scipy as sp
import scipy.stats as st
import matplotlib as mpl
import matplotlib.pyplot as plt
import analytics
import pandas as pd

#import plotly.plotly as py
#import plotly.graph_objs as pgo
!ls

2015-07-14.csv		  analytics.py	 collaboration_scoring_scenarios.csv
analytics-notebook.ipynb  analytics.pyc  Screenshot


In [4]:
with open("2015-07-14.csv") as baseFile:
    csvr = csv.reader(baseFile)
    records=[]
    for line in csvr:
        records.append(line)
print(records[0])
records=records[1:]

['time', 'hangout_id', 'participant_id', 'volume', 'recorder_id']


In [9]:
print(records[0])
print(len(records))
#filter(lambda x:(operator.gt(int(x[3]),0)),records[1:])

['1436906515049', 'hangout89FB9D61_ephemeral.id.google.com^902c79f4a20768', 'hangout89FB9D61_ephemeral.id.google.com^902c79f4a20768', '2', 'hangout89FB9D61_ephemeral.id.google.com^902c79f4a20768 ']
23313


In [10]:
#Testing the data set as a whole. This gets some basic sanity checks done.
get = lambda n: ([k[n] for k in records])
#Verification that all times are ints, and displaying the offending lines otherwise.
#for k in records:
#    try:
#        int(k[0])
#    except Exception,e:
#        print(k)
times = list(map(int,get(0)))
hangout_ids = set(get(1))
participant_ids = set([k[2] for k in records])#Not an int.
vols = list(map(int,get(3)))
vol_range = set(vols)

In [11]:
plt.figure("Sanity Check 1")
plt.subplot(211)
plt.plot(times)
plt.subplot(212)
plt.plot(vols)
plt.figure("Sanity Check 2")
plt.subplot(211)
plt.plot(times,vols)
plt.subplot(212)

histo_getter = lambda n,l: len([m for m in l if m==n])
lnhg = lambda n,l: (math.log(histo_getter(n,l)))
vg = lambda n:lnhg(n,vols)
vol_histogram = [vg(0),vg(1),vg(2),vg(3),vg(4),vg(5)]
plt.title("Natural log of frequencies of different volumes.")
plt.plot([0,1,2,3,4,5],vol_histogram)

In [13]:
plt.show()

In [27]:
#Now we can compare between participants.
bpr = analytics.by_participant(records)
#This is a list, for each participant
print(len(records))
print(len(bpr))
print(sum(map(len,bpr)))

23313
14
23313


In [5]:
# Now we compare between hangouts.
compressedRecords=analytics.longer(records)

In [10]:
#Trivial check of collaboration scores in different hangouts.
b = analytics.by_hangout(records)
list(map(lambda x:(analytics.n_h_index(x)),b))
list(map(lambda x:(analytics.pause_count(x,1000,50000)),b))
list(map(analytics.crosstalk,b))

[0.8543689320388349, 0.9658444985789687, 0.9246978617911373, 0.90625, 0.0]

In [11]:
x=analytics.track_changes(compressedRecords,analytics.n_h_index)
x
plt.figure("Herfindahl Index over time.")
plt.plot(x)
plt.show()